# Named Entity Recognition #
Methods developed for various template generation. To begin the demo please run the commands below to clone the repository.

For DSTC10 information, please visit: https://dstc10.dstc.community/tracks

For track2 dataset, please visit: https://github.com/alexa/alexa-with-dstc10-track2-dataset

In [ ]:
!git clone  #

In [ ]:
!pip install -r requirements.txt

### Baby Trie ###
Baby Trie is a trie data structure built for named entity recognition. It is built using a tree structure and with a given knowledge base. After intialization, the trie takes in a loaded json file as argument to create its tree structure. 

We define an entity $P$ with word length $n$. To insert an entity into the trie, we create a child node to the root containing $P[0]$, the first word. We then traverse down to that node and create another child node $P[1]$. This is repeated until $P[n-1]$ is inserted.



*Note the Baby Trie stucture is programmed for DSTC datasets. See track 2 dataset for knowledge base file.*

In [ ]:
# Load our json knowledge base file into a dictionary
import json

knowledge_base=open('../resources/knowledge.json','r')
dic=json.load(knowledge_base)
knowledge_base.close()

In [ ]:
# Build trie from loaded json file
from BabyTrie import BabyTrie

trie = BabyTrie()
trie.initialize(dic)

# List of all possible first match, P[0]s
trie.root.children

We distinguish if something has a complete match by determining if they are a leaf or not. If a given query has multiple names matched within a range of index, the trie would recognize all of them, but return the longest matched.

In [ ]:
# Demonstration, A and B Guest House is a restaurant name
query = "Can I bring my pet to A and B Guest House?"

template, mapping = gettemplate_wmap(trie, query)

print("Original input query:\t{q}\nGenerated template:\t{t}\nGenerated mapping:\t{m}".format(q=query,t=template,m=mapping))

Original input query:	Can I bring my pet to A and B Guest House?
Generated template:	Can I bring my pet to <hotel-0>?
Generated mapping:	{'<hotel-0>': 'A and B Guest House'}


In [ ]:
# Demonstration, Pier 39 is an attraction name
query = "Are there restrictions for pier thirty nine?"

template, mapping = gettemplate_wmap(trie, query)

print("Original input query:\t{q}\nGenerated template:\t{t}\nGenerated mapping:\t{m}".format(q=query,t=template,m=mapping))

Original input query:	Are there restrictions for pier thirty nine?
Generated template:	Are there restrictions for <attraction-100123>?
Generated mapping:	{'<attraction-100123>': 'pier thirty nine'}


In [ ]:
# If a chained restaurant is given, the trie matches differently, appending all possible chained entities.
query = "What food is offered at Rooster and rice?"

template, mapping = gettemplate_wmap(trie, query)

print("Original input query:\t{q}\nGenerated template:\t{t}\nGenerated mapping:\t{m}".format(q=query,t=template,m=mapping))

Original input query:	What food is offered at Rooster and rice?
Generated template:	What food is offered at <restaurant-120353|120354|120355|120356|120357>?
Generated mapping:	{'<restaurant-120353|120354|120355|120356|120357>': 'Rooster and rice'}


### Performance ###
As noticed, the trie can match Pier 39 with pier thirty nine. The trie uses num2words and a custom converttext2num function to generate all permutations of the entity and insert them all into the trie.

This is important as ASR queries contain only pronunciated words, but the knowledge base represents them as numbers.

Another feature added was Damerau Levenshtein matching. 

After data analysis, a common mismatch happens due to noise in a query. Example:

*There is boudin ba bakery and cafe.*

Most likely, the speaker stuttered "ba". As the way traditional trie structures are, they match in a reserved manner. However, in this context, we want our trie to allow some noise to *pass-through* and match an entity.

We introduce the damerau levenshtein distance matching. It is a string metric for measuring the edit distance between two sequences. To put it simply, the DL-distance between two words is the minimum number of operations (consisting of insertions, deletions, or substitutions of a single character, or transposition of two adjacent characters) required to change one word into the other.

Read more about it here: https://en.wikipedia.org/wiki/Damerau%E2%80%93Levenshtein_distance


In [ ]:
# The fastDamerau-Levenshtein offers flexible matching.
query = "There is boudin ba bakery and cafe."

template, mapping = gettemplate_wmap(trie, query)

print("Original input query:\t{q}\nGenerated template:\t{t}\nGenerated mapping:\t{m}".format(q=query,t=template,m=mapping))

Original input query:	There is boudin ba bakery and cafe.
Generated template:	There is <restaurant-120053>.
Generated mapping:	{'<restaurant-120053>': 'boudin ba bakery and cafe'}


Below are some other examples.

In [ ]:
# Knowledge Base entity: S. W. hotel
query = "Can you give me the direction for sw hotel?"

template, mapping = gettemplate_wmap(trie, query)

print("Original input query:\t{q}\nGenerated template:\t{t}\nGenerated mapping:\t{m}".format(q=query,t=template,m=mapping))

Original input query:	Can you give me the direction for sw hotel?
Generated template:	Can you give me the direction for <hotel-110167>?
Generated mapping:	{'<hotel-110167>': 'sw hotel'}


In [ ]:
# Knowledge Base entity: Saigon City
query = "I need to make a reservation for siagon city."

template, mapping = gettemplate_wmap(trie, query)

print("Original input query:\t{q}\nGenerated template:\t{t}\nGenerated mapping:\t{m}".format(q=query,t=template,m=mapping))

Original input query:	I need to make a reservation for siagon city.
Generated template:	I need to make a reservation for <restaurant-19261>.
Generated mapping:	{'<restaurant-19261>': 'siagon city'}


Other methods implemented include:
* disfluency removal
* restricted insertion (for area mismatches)
* phonetic conversion (for error correction on query and elasticsearch)